In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***Minor Data Quality Issue*** \
Before loading the data into Python, it was observed in Excel that the entries in the barcode column of both the PRODUCTS_TAKEHOME and TRANSACTIONS_TAKEHOME csv files which had either "0" or "00" at the start of the barcode were preceded with an apostrophe (" ' ") mark. And also, when the barcodes were looked up online to check for their legitimacy, the barcodes that started with two zeros ("00") with a preceding apostrophe mark were not present in the database. While not all the barcodes starting with "00" were tested, but whichever were looked up, they turned out to be invalid

It was also observed that numeric-like strings are automatically converted into a float or integer type when being loaded into Python and the leading zeros in the number are removed automatically.

Let's see if thats true:

In [3]:
product = pd.read_csv('/content/drive/My Drive/PRODUCTS_TAKEHOME.csv')
product['BARCODE'].dtype

dtype('float64')

In [4]:
product['BARCODE'].head(20)

,BARCODE
0,7.964944e+11
1,2.327801e+10
2,4.618178e+11
3,3.500047e+10
4,8.068109e+11
5,6.626585e+11
6,6.177376e+11
7,7.501839e+12
8,7.545013e+10
9,NaN


It can be observed that the barcode column has been converted into a float type and no observation starts with a zero like in the actual records. Let's correct that! Let us reload the data considering the barcode column in both the above mentioned csv files as strings to retain the zeros.

Let us import the three data files into colab!

In [5]:
users = pd.read_csv('/content/drive/My Drive/USER_TAKEHOME.csv')
transact = pd.read_csv('/content/drive/My Drive/TRANSACTION_TAKEHOME.csv',dtype={'BARCODE': str})
product = pd.read_csv('/content/drive/My Drive/PRODUCTS_TAKEHOME.csv', dtype={'BARCODE':str})

The data has been reloaded with the barcode as a string datatype.
This process is being carried out to retain the initial zeros in the barcodes to ensure their validity.

# Users data analysis

Checking the data types of the various columns in the users table

In [6]:
users.dtypes

,0
ID,object
CREATED_DATE,object
BIRTH_DATE,object
STATE,object
LANGUAGE,object
GENDER,object


In [7]:
users.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female


Let us check if the columns contain any lists, arrays or dictionaries.

In [8]:
def check_column_datastructures(column):
    return any(isinstance(x, (list, dict, np.ndarray)) for x in column)

# Applying the function to each column
columns_with_structures = {col: check_column_datastructures(users[col]) for col in users.columns}

# Displaying which columns contain arrays, lists, or dictionaries
print("Columns containing arrays, lists, or dictionaries:")
for col, contains_structure in columns_with_structures.items():
    if contains_structure:
        print(f"{col} contains complex structures")

Columns containing arrays, lists, or dictionaries:


In [9]:
users.describe(include = 'all')

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
count,100000,100000,96325,95188,69492,94108
unique,100000,99942,54721,52,2,11
top,5ef3b4f17053ab141787697d,2023-01-12 18:30:15.000 Z,1970-01-01 00:00:00.000 Z,TX,en,female
freq,1,2,1272,9028,63403,64240



***Observations*** \
1)It can be observed that the 'ID' column has all unique values, which is good, so there are no duplicate rows.\
2) There are some dates in the "CREATED_DATE" column which seem to be recurring. We might get a better understanding about the repeated values. \
3) There are a lot of missing values in the language column and also its better to include more unique languages.\
4) There are some missing birth date values and further investigation is needed.\
5) Coming to the genders and the states columns, there seem to be 11 unique values and 52 unique values respectively.

**Converting the created date and the birth date into date-time objects**

In [10]:
users['BIRTH_DATE'] = pd.to_datetime(users['BIRTH_DATE'])
users['CREATED_DATE'] = pd.to_datetime(users['CREATED_DATE'])

# Make 'datetime_no_tz' timezone-aware by localizing it to UTC
users['BIRTH_DATE'] = users['BIRTH_DATE'].dt.tz_convert('UTC')
users['CREATED_DATE'] = users['CREATED_DATE'].dt.tz_convert('UTC')

users.dtypes

,0
ID,object
CREATED_DATE,"datetime64[ns, UTC]"
BIRTH_DATE,"datetime64[ns, UTC]"
STATE,object
LANGUAGE,object
GENDER,object


In [11]:
users.describe(include = 'all')

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
count,100000,100000,96325,95188,69492,94108
unique,100000,NaN,NaN,52,2,11
top,5ef3b4f17053ab141787697d,NaN,NaN,TX,en,female
freq,1,NaN,NaN,9028,63403,64240
mean,NaN,2022-01-07 05:31:20.864859648+00:00,1984-09-02 02:39:04.710417920+00:00,NaN,NaN,NaN
min,NaN,2014-04-18 23:14:55+00:00,1900-01-01 00:00:00+00:00,NaN,NaN,NaN
25%,NaN,2020-12-01 21:16:19+00:00,1974-03-04 00:00:00+00:00,NaN,NaN,NaN
50%,NaN,2022-03-07 01:03:37+00:00,1985-10-25 00:00:00+00:00,NaN,NaN,NaN
75%,NaN,2023-01-30 13:47:44.500000+00:00,1998-02-02 05:00:00+00:00,NaN,NaN,NaN
max,NaN,2024-09-11 17:59:15+00:00,2022-04-03 07:00:00+00:00,NaN,NaN,NaN


***Observations*** \
The earliest year in the birthdate column is 1900 which seems to be very early. This might cause a problem when age is used as a factor in an analysis for drawing business insights. How much it can affect can only be understood by further investigation.

**Checking for null values values:**

In [12]:
users.isna().sum()

,0
ID,0
CREATED_DATE,0
BIRTH_DATE,3675
STATE,4812
LANGUAGE,30508
GENDER,5892


In [13]:
#Lets check for percentages
users_isna = users.isna().sum()
users_isna/users.shape[0]

,0
ID,0.00000
CREATED_DATE,0.00000
BIRTH_DATE,0.03675
STATE,0.04812
LANGUAGE,0.30508
GENDER,0.05892


**Looking more into birth_date and created_date**

**Checking for records where the birthdate is after the created date**

In [14]:
# Comparing Birth dates and created dates and storing it in a new variable
users['BD<CD'] = users['BIRTH_DATE'].dt.date <= users['CREATED_DATE'].dt.date

users['BD<CD'].value_counts()

,count
BD<CD,
True,96324
False,3676


It seems that there are around 3700 records where the birt_date is after the created date. This might because of the null values in the birthddate column, so let us filter them out and see.

In [15]:
users_cd_b4_bd = users[(users['BD<CD']==False) & (~users['BIRTH_DATE'].isna())]
users_cd_b4_bd.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER,BD<CD
41974,5f31fc048fa1e914d38d6952,2020-08-11 02:01:41+00:00,2020-10-02 15:27:28+00:00,CA,NaN,NaN,False


***Minor Data Quality Issue*** \
There seems to be one record where the birthdate is after the created date. Let us remove that record!

In [16]:
users.drop(users[users['ID'] == '5f31fc048fa1e914d38d6952'].index, inplace=True)

In [17]:
users.shape

(99999, 7)

Let us drop the additional boolean column as it is no longer needed

In [18]:
users = users.drop('BD<CD', axis=1)

In [19]:
users.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54+00:00,2000-08-11 00:00:00+00:00,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55+00:00,2001-09-24 04:00:00+00:00,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18+00:00,1994-10-28 00:00:00+00:00,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22+00:00,NaT,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50+00:00,1972-03-19 00:00:00+00:00,PA,en,female


In [20]:
#Checking the number of occurences of each date in the "BIRTH_DATE" column
users['BIRTH_DATE'].value_counts()

,count
BIRTH_DATE,
1970-01-01 00:00:00+00:00,1272
1979-12-11 08:00:00+00:00,63
2000-12-12 00:00:00+00:00,28
2000-12-31 00:00:00+00:00,23
2001-01-01 00:00:00+00:00,16
...,...
2004-02-21 08:00:00+00:00,1
1956-02-09 05:00:00+00:00,1
1974-07-13 00:00:00+00:00,1


It can be observed that, a lot of users chose 1st January 1970 as their birth date. This might be because, the app's default date is set to 1st January 1970.

Let us look more into the records where the birthdate is missing.

In [21]:
users_bdate_miss = users[users['BIRTH_DATE'].isna()]
users_bdate_miss.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22+00:00,NaT,NC,en,NaN
25,6568861a9cb6a4f05dc9ccda,2023-11-30 12:54:50+00:00,NaT,NC,en,NaN
28,64fa0e8c791f8099cbbd1d27,2023-09-07 17:55:24+00:00,NaT,NaN,en,NaN
37,655fe9a1dd170fff1657a048,2023-11-24 00:09:05+00:00,NaT,LA,en,NaN
59,65206ec67933bcf6ee794eaf,2023-10-06 20:32:06+00:00,NaT,TX,en,NaN


Let us focus on the individual columns. What if the birthdates are missing during certain time periods?

In [22]:
# Extracting the month and year of the created_date and counting the number of created accounts with missing birth dates monthly.
users_bdate_miss.loc[:,'CREATED_MONTH'] = users_bdate_miss['CREATED_DATE'].dt.to_period('M')
users_bdate_miss['CREATED_MONTH'].value_counts().sort_index()

<ipython-input-22-8b315ac65b83>:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  users_bdate_miss.loc[:,'CREATED_MONTH'] = users_bdate_miss['CREATED_DATE'].dt.to_period('M')
<ipython-input-22-8b315ac65b83>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_bdate_miss.loc[:,'CREATED_MONTH'] = users_bdate_miss['CREATED_DATE'].dt.to_period('M')


,count
CREATED_MONTH,
2021-11,1
2022-01,1
2023-04,6
2023-05,18
2023-06,46
2023-07,145
2023-08,249
2023-09,279
2023-10,342


Let us check for the created dates month wise in the complete dataset to get an idea regarding the overall issue

In [23]:
#Extracting the month and year of the created_date and counting the number of accounts created every month
users.loc[:,'CREATED_MONTH'] = users['CREATED_DATE'].dt.to_period('M')
users['CREATED_MONTH'].value_counts().sort_index()

<ipython-input-23-44ac412c61d7>:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  users.loc[:,'CREATED_MONTH'] = users['CREATED_DATE'].dt.to_period('M')


,count
CREATED_MONTH,
2014-04,2
2014-05,13
2014-07,4
2014-08,2
2014-09,3
...,...
2024-05,1146
2024-06,1260
2024-07,2037


In [24]:
users['CREATED_MONTH'].value_counts().sort_index().tail(35)

,count
CREATED_MONTH,
2021-11,1630
2021-12,1703
2022-01,1869
2022-02,1611
2022-03,1817
2022-04,1813
2022-05,1916
2022-06,2696
2022-07,3190


In [25]:
#Comparing the counts during the common months in both the cases.
users_crdate_monthwise = users['CREATED_MONTH'].value_counts().sort_index().tail(35)
users_crdate_bdmiss_mwise = users_bdate_miss['CREATED_MONTH'].value_counts().sort_index()
created_date_percent = (users_crdate_bdmiss_mwise/users_crdate_monthwise)*100
created_date_percent

,count
CREATED_MONTH,
2021-11,0.061350
2021-12,NaN
2022-01,0.053505
2022-02,NaN
2022-03,NaN
2022-04,NaN
2022-05,NaN
2022-06,NaN
2022-07,NaN


***Major Data Quality issue*** \
It seems that the problem of missing birthdate started in November, 2021 and the problem worsened over the following months. While in 2022 the records are fine, but in 2023, starting from 0.5% of created accounts having a missing birthdate, it reached a staggering 55.75% in 2023 December. This means that for more than half of the accounts created in December of 2023, the birth date of the user is missing! Especially, more than 45% of accounts created in November and December of 2023 and January of 2024, have missing birth_date information. This affects age-based analysis adversely!   

In [26]:
users_bdate_miss.isna().sum()

,0
ID,0
CREATED_DATE,0
BIRTH_DATE,3675
STATE,2116
LANGUAGE,32
GENDER,3601
CREATED_MONTH,0


Another observation here is that, out of all the records where birthdate is missing, 97.98% of them have missing gender information and one reason might be that both the gender and the birthdate details were asked on the same page of the application.

In [27]:
#Lets check the breakup of missing values in the 'STATE' column where the birth_date values are null.
users_bdate_miss['STATE'].isna().value_counts()

,count
STATE,
True,2116
False,1559


In [28]:
#Lets check the breakup of missing values in the 'LANGUAGE' column where the birth_date values are null.
users_bdate_miss['LANGUAGE'].isna().value_counts()

,count
LANGUAGE,
False,3643
True,32


Two interesting observations here are:\
1) Out of 5892 missing STATE values, 2116 missing values are present where there are missing 'birth_date' values. That is 36% of the total missing values.\
2) Though there are almost 30,500 missing values in "LANGUAGE" column, only 32 values are missing where the birth dates are missing. That is roughly 1.04% of the total missing values.


Let us drop the 'CREATED_MONTH' column

In [29]:
users = users.drop('CREATED_MONTH', axis=1)

**Lets check for records where the difference between the birth dates and created dates is more than 100.**

In [30]:
users_filtered = users[users['CREATED_DATE'].notna() & users['BIRTH_DATE'].notna()]

In [31]:
users_filtered.loc[:,'year difference'] = (users_filtered['CREATED_DATE'] - users_filtered['BIRTH_DATE']).dt.days/365.25
users_filtered.head()

<ipython-input-31-dcc8d94733c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_filtered.loc[:,'year difference'] = (users_filtered['CREATED_DATE'] - users_filtered['BIRTH_DATE']).dt.days/365.25


,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER,year difference
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54+00:00,2000-08-11 00:00:00+00:00,CA,es-419,female,19.868583
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55+00:00,2001-09-24 04:00:00+00:00,PA,en,female,19.277207
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18+00:00,1994-10-28 00:00:00+00:00,FL,es-419,female,28.588638
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50+00:00,1972-03-19 00:00:00+00:00,PA,en,female,51.608487
5,5fe2b6f3ad416a1265c4ab68,2020-12-23 03:18:11+00:00,1999-10-27 04:00:00+00:00,NY,en,female,21.155373


In [32]:
users_age100 = users_filtered[users_filtered['year difference']>100]
users_age100.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER,year difference
3656,62be5974baa38d1a1f6b6725,2022-07-01 02:18:28+00:00,1903-01-01 05:00:00+00:00,PA,en,male,119.493498
5878,60ac6acc79ed9200a6ebc24a,2021-05-25 03:11:08+00:00,1901-05-25 00:00:00+00:00,UT,NaN,female,120.000000
6499,60a6e4af3369535cb6c4c89d,2021-05-20 22:37:35+00:00,1901-10-13 00:00:00+00:00,CA,NaN,male,119.600274
11146,608601a7b14c7f309d219fc6,2021-04-25 23:56:23+00:00,1904-10-28 19:56:38+00:00,FL,es-419,female,116.490075
14270,59bc4fd3e4b03f96c479b7e6,2017-09-15 22:10:27+00:00,1905-07-28 00:00:00+00:00,CA,en,female,112.134155


In [33]:
users_age100.describe(include= 'all')

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER,year difference
count,56,56,56,50,22,53,56.000000
unique,56,NaN,NaN,25,2,5,NaN
top,62be5974baa38d1a1f6b6725,NaN,NaN,CA,en,female,NaN
freq,1,NaN,NaN,6,20,28,NaN
mean,NaN,2021-02-15 14:13:51.464285696+00:00,1907-12-05 10:18:15.928571392+00:00,NaN,NaN,NaN,113.198103
min,NaN,2015-02-20 19:57:01+00:00,1900-01-01 00:00:00+00:00,NaN,NaN,NaN,100.624230
25%,NaN,2020-06-05 21:09:18.750000128+00:00,1901-10-02 06:00:00+00:00,NaN,NaN,NaN,107.470910
50%,NaN,2021-05-31 01:33:34.500000+00:00,1906-02-25 19:00:00+00:00,NaN,NaN,NaN,115.226557
75%,NaN,2022-02-20 17:58:30.500000+00:00,1912-09-10 05:45:00+00:00,NaN,NaN,NaN,119.518138
max,NaN,2024-09-11 17:52:47+00:00,1920-06-26 04:00:00+00:00,NaN,NaN,NaN,121.251198


There are 56 observations where the age is more than 100 with the maximum being 121 years. This can be due to a wrong input from the users. The next step that we can see is that how many of these accounts are a part of the transactions.

In [34]:
#Lets rename the 'user_id' column name to match the one in users table
transact.rename(columns={'USER_ID' : 'ID'},inplace=True)
#

In [35]:
common_ids = pd.merge(users_age100,transact, on='ID', how='inner')
common_ids.shape

(0, 14)

From the above result, it can be seen that none of the ids with age more than 100 are present in the transact table. So we can ignore them.

Earlier it was found that 'CREATED_DATE' had some repeated values along with the same time stamp. Let us look more into it.

In [36]:
users['CREATED_DATE'].value_counts()

,count
CREATED_DATE,
2021-08-06 14:51:13+00:00,2
2019-08-28 02:21:44+00:00,2
2024-04-11 02:56:41+00:00,2
2024-03-11 17:03:02+00:00,2
2024-02-25 20:43:59+00:00,2
...,...
2024-08-25 03:30:41+00:00,1
2021-03-12 13:08:04+00:00,1
2024-07-24 19:33:12+00:00,1


There seem to be multiple instances where two users have the same created_date and time. Let us check more about it

In [37]:
users_crdt_twice = users[users['CREATED_DATE'].map(users['CREATED_DATE'].value_counts()) == 2]
users_crdt_twice.shape


(116, 6)

In [38]:
users_crdt_twice.groupby(['CREATED_DATE','STATE','BIRTH_DATE'])['ID'].apply(list).head(15)

CREATED_DATE               STATE  BIRTH_DATE               
2019-08-25 02:02:11+00:00  AR     1975-08-12 05:00:00+00:00    [5d61ec22fe79a7584c9b573c]
                           VA     1972-06-07 00:00:00+00:00    [5d61ec231ddc4058bd9a6233]
2019-08-28 02:21:44+00:00  MS     1990-10-15 05:00:00+00:00    [5d65e537d09cf73c7b6a1585]
                           NH     1975-05-15 00:00:00+00:00    [5d65e5381ddc403b76f4dc72]
2020-01-08 01:42:14+00:00  CT     1979-02-07 05:00:00+00:00    [5e153376128c2c120e86e57f]
2020-02-16 17:04:11+00:00  AR     1990-12-03 06:00:00+00:00    [5e49760aacedab1335b03b89]
                           GA     1972-04-15 05:00:00+00:00    [5e49760b164813133fc63ac2]
2020-04-29 02:24:48+00:00  CA     1992-10-12 00:00:00+00:00    [5ea8e56f2244e629eacf9b09]
                           UT     1978-06-07 06:00:00+00:00    [5ea8e56f2244e629eacf9b07]
2020-05-04 01:01:40+00:00  CA     1979-03-08 08:00:00+00:00    [5eaf6974cefff2142582eab6]
                           MI     1999-03-04 05:00:00+00:00    [5eaf6973787646145f3a1ccf]
2020-08-29 22:21:47+00:00  KY     2008-07-08 04:00:00+00:00    [5f4ad4fa88785515f05bb940]
                           MI     2000-12-09 00:00:00+00:00    [5f4ad4fac715ef1783cbf336]
2020-10-25 16:35:07+00:00  GA     1951-03-09 05:00:00+00:00    [5f95a93b55f095164fe322d9]
                           MO     1964-08-11 05:00:00+00:00    [5f95a93a16ab0a167bd786d8]
Name: ID, dtype: object

I wanted to check if the accounts were created had the same state, but doesnt seem to be the case. Let us check for the time period during which the repetitions occured.

In [39]:
users_crdt_twice_og = users_crdt_twice.drop_duplicates(subset=['CREATED_DATE'], keep='first')
users_crdt_twice_og.loc[:,'CREATED_MONTH'] = users_crdt_twice_og['CREATED_DATE'].dt.to_period('M')
users_crdt_twice_og['CREATED_MONTH'].value_counts().sort_index()

<ipython-input-39-2f98745bbf34>:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  users_crdt_twice_og.loc[:,'CREATED_MONTH'] = users_crdt_twice_og['CREATED_DATE'].dt.to_period('M')
<ipython-input-39-2f98745bbf34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_crdt_twice_og.loc[:,'CREATED_MONTH'] = users_crdt_twice_og['CREATED_DATE'].dt.to_period('M')


,count
CREATED_MONTH,
2019-08,2
2020-01,1
2020-02,1
2020-04,1
2020-05,1
2020-08,1
2020-10,2
2020-12,2
2021-01,2


The above result doesnt give much insight into the issue.

**Focus on the states and the gender columns:**

In [40]:
#Listing out the unique values of in the 'STATE' column
users['STATE'].unique()

array(['CA', 'PA', 'FL', 'NC', 'NY', 'IN', nan, 'OH', 'TX', 'NM', 'PR',
       'CO', 'AZ', 'RI', 'MO', 'NJ', 'MA', 'TN', 'LA', 'NH', 'WI', 'IA',
       'GA', 'VA', 'DC', 'KY', 'SC', 'MN', 'WV', 'DE', 'MI', 'IL', 'MS',
       'WA', 'KS', 'CT', 'OR', 'UT', 'MD', 'OK', 'NE', 'NV', 'AL', 'AK',
       'AR', 'HI', 'ME', 'ND', 'ID', 'WY', 'MT', 'SD', 'VT'], dtype=object)

In [41]:
#Listing out the number of occurences of each unique value and ordering them
users['STATE'].value_counts()

,count
STATE,
TX,9028
FL,8921
CA,8588
NY,5703
IL,3794
GA,3718
PA,3644
OH,3519
NC,3399


It can be noticed that, number of rows with 'nan' values rank 5th when all the states are ranked with respect to the occurrences. Lets check out the row details where the states column is null.

In [42]:
users[users['STATE'].isnull()].describe()


,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
count,4812,4812,2696,0,2539,1403
unique,4812,NaN,NaN,0,2,9
top,608823160155c475fd885921,NaN,NaN,NaN,en,female
freq,1,NaN,NaN,NaN,2247,704
mean,NaN,2022-05-29 22:52:26.768910848+00:00,1979-06-06 15:35:06.841246272+00:00,NaN,NaN,NaN
min,NaN,2014-04-18 23:14:55+00:00,1900-01-01 00:00:00+00:00,NaN,NaN,NaN
25%,NaN,2021-02-26 17:11:27.750000128+00:00,1970-01-01 00:00:00+00:00,NaN,NaN,NaN
50%,NaN,2022-09-03 19:04:10.500000+00:00,1970-01-01 00:00:00+00:00,NaN,NaN,NaN
75%,NaN,2023-12-04 23:12:50+00:00,1990-03-11 22:56:00.750000+00:00,NaN,NaN,NaN
max,NaN,2024-09-11 17:59:15+00:00,2022-04-03 07:00:00+00:00,NaN,NaN,NaN


In [43]:
#Listing out the unique values of in the 'GENDER' column
users['GENDER'].unique()

array(['female', nan, 'male', 'non_binary', 'transgender',
       'prefer_not_to_say', 'not_listed', 'Non-Binary', 'unknown',
       'not_specified', "My gender isn't listed", 'Prefer not to say'],
      dtype=object)

In [44]:
#Listing out the number of occurences of each unique value and ordering them
users['GENDER'].value_counts()

,count
GENDER,
female,64240
male,25829
transgender,1772
prefer_not_to_say,1350
non_binary,473
unknown,196
not_listed,180
Non-Binary,34
not_specified,28


It can be noticed that, number of rows with 'nan' values rank 3th when all the genders are ranked with respect to the occurrences. \
***Minor Data Quality Issue*** \
There are two categories in the gender column which mean the same those are "prefer_not_to_say" and "Prefer not to say". \
Lets check out the row details where the states column is null.

In [45]:
users[users['GENDER'].isnull()].describe()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
count,5891,5891,2290,2482,4011,0
unique,5891,NaN,NaN,52,2,0
top,658a306e99b40f103b63ccf8,NaN,NaN,FL,en,NaN
freq,1,NaN,NaN,246,3516,NaN
mean,NaN,2022-09-03 09:59:40.269054720+00:00,1975-08-30 08:09:19.650655008+00:00,NaN,NaN,NaN
min,NaN,2014-04-18 23:14:55+00:00,1900-01-01 00:00:00+00:00,NaN,NaN,NaN
25%,NaN,2021-05-11 17:38:24+00:00,1970-01-01 00:00:00+00:00,NaN,NaN,NaN
50%,NaN,2023-09-23 16:39:07+00:00,1970-01-01 00:00:00+00:00,NaN,NaN,NaN
75%,NaN,2024-01-05 13:55:41+00:00,1982-01-03 05:00:00+00:00,NaN,NaN,NaN
max,NaN,2024-09-11 17:59:15+00:00,2022-03-01 05:00:00+00:00,NaN,NaN,NaN


**Focus on the Language column**

In [46]:
users_nol = users[users['LANGUAGE'].isna()]
users_nol.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
10,608823160155c475fd885921,2021-04-27 14:43:34+00:00,1981-04-27 04:00:00+00:00,NaN,NaN,male
12,6087490b0155c475fd855961,2021-04-26 23:13:15+00:00,1998-04-29 04:00:00+00:00,PA,NaN,female
13,5f07489d2def33149571ad98,2020-07-09 16:41:02+00:00,1985-10-21 05:00:00+00:00,TX,NaN,female
15,5b03790ce4b0dbe82bdc768b,2018-05-22 01:57:32+00:00,1982-11-01 00:00:00+00:00,PR,NaN,female
17,5f7b4ff3294ea2162cc50d2e,2020-10-05 16:55:15+00:00,1999-09-21 00:00:00+00:00,CO,NaN,female


In [47]:
#Lets check for the states where most of the missing data is there
users_nol['STATE'].value_counts()

,count
STATE,
CA,3009
TX,2846
FL,2220
NY,1568
GA,1055
IL,1038
NC,943
PA,868
OH,866


# Products Data Analysis

In [48]:
product.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,796494407820
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,023278011028
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,461817824225
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,035000466815
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,0806810850459


**Checking the datatypes**

In [49]:
product.columns

Index(['CATEGORY_1', 'CATEGORY_2', 'CATEGORY_3', 'CATEGORY_4', 'MANUFACTURER',
       'BRAND', 'BARCODE'],
      dtype='object')

In [50]:
product.shape

(845552, 7)

**Checking for duplicate records**

In [51]:
product_dupe = product[product.duplicated()]
product_dupe.shape

(57, 7)

***Data Quality Issue*** \
There are few duplicate rows present. They can be removed.



In [52]:
#Removing the duplicate records
product_real = product.drop_duplicates()

In [53]:
product_real.shape

(845495, 7)

Let us check if any of the columns contain an array, list or a dictionary.

In [54]:
def check_column_types(column):
    return any(isinstance(x, (list, dict, np.ndarray)) for x in column)

# Apply the function to each column
columns_with_structures = {col: check_column_types(product_real[col]) for col in product_real.columns}

# Display which columns contain arrays, lists, or dictionaries
print("Columns containing arrays, lists, or dictionaries:")
for col, contains_structure in columns_with_structures.items():
    if contains_structure:
        print(f"{col} contains complex structures")

Columns containing arrays, lists, or dictionaries:


In [55]:
product_real.describe(include='all')

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
count,845384,844073,784932,67455,619021,619023,841527
unique,27,121,344,127,4354,8122,841525
top,Health & Wellness,Candy,Confection Candy,Lip Balms,PLACEHOLDER MANUFACTURER,REM BRAND,017000329260
freq,512687,121034,56964,9737,86902,20813,2


***Minor Data Quality Issue*** \
There seems to be 2 barcodes that got repeated twice, let us check a little more about that!


In [56]:
product_real['BARCODE'].value_counts()

,count
BARCODE,
017000329260,2
052336919068,2
796494407820,1
850010304229,1
820645007123,1
...,...
433565028514,1
0603029132914,1
072992040241,1


In [57]:
product_real[(product_real['BARCODE']=='017000329260') | (product_real['BARCODE']=='052336919068')]

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
28421,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,SCHWARZKOPF,052336919068
213340,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,SCHWARZKOPF,017000329260
304021,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,GÖT2B,017000329260
709607,Health & Wellness,Hair Care,Hair Color,NaN,HENKEL,GÖT2B,052336919068


It seems that while the brand is different, the barcode is the same. Though not sure about the reason, I would flag it as a minor issue.

Let us just see if this barcode is present in the transaction dataset or not.

In [58]:
transact[(transact['BARCODE']=='017000329260') | (transact['BARCODE']=='052336919068')]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,ID,BARCODE,FINAL_QUANTITY,FINAL_SALE


Good news! It is not present.

**Checking for the presence of Null Values**

In [59]:
product_real.isna().sum()

,0
CATEGORY_1,111
CATEGORY_2,1422
CATEGORY_3,60563
CATEGORY_4,778040
MANUFACTURER,226474
BRAND,226472
BARCODE,3968


In [60]:
#Lets check for percentages
product_isna = product_real.isna().sum()
product_isna/product_real.shape[0]

,0
CATEGORY_1,0.000131
CATEGORY_2,0.001682
CATEGORY_3,0.071630
CATEGORY_4,0.920218
MANUFACTURER,0.267860
BRAND,0.267857
BARCODE,0.004693


**Observations** \
1) 92.02% percent of the category 4 are missing values.\
2) There are almost equal amount of missing values in both Manufacturer column and the brand column which is 26.78%.\
3) The most concerning observation is that there are several products whose barcodes are missing. Further investigation is required to understand the implications.



**Focus on 'Category_1' column**

In [61]:
#Listing out the unique categories in the column
product_real['CATEGORY_1'].unique()

array(['Health & Wellness', 'Snacks', 'Beverages', 'Pantry', 'Alcohol',
       'Apparel & Accessories', 'Restaurant', 'Needs Review', 'Dairy',
       'Home & Garden', nan, 'Household Supplies', 'Meat & Seafood',
       'Deli & Bakery', 'Sporting Goods', 'Produce', 'Office & School',
       'Frozen', 'Arts & Entertainment', 'Animals & Pet Supplies',
       'Electronics', 'Beauty', 'Toys & Games', 'Mature',
       'Vehicles & Parts', 'Baby & Toddler', 'Luggage & Bags', 'Media'],
      dtype=object)

In [62]:
product_real['CATEGORY_1'].value_counts()

,count
CATEGORY_1,
Health & Wellness,512687
Snacks,324798
Beverages,3990
Pantry,870
Apparel & Accessories,846
Dairy,602
Needs Review,547
Alcohol,502
Home & Garden,115


We can see that there is a category called 'Needs Review' in the 'CATEGORY_1' column. Lets analyze further:

In [63]:
#Checking the number of rows where 'CATEGORY_1' is 'Needs Review'
product_real[product_real['CATEGORY_1']=='Needs Review'].shape

(547, 7)

There are 547 items which are labeled as 'Needs Review' in 'CATEGORY_1' column.

In [64]:
product_real[product_real['CATEGORY_1']=='Needs Review'].isna().sum()

,0
CATEGORY_1,0
CATEGORY_2,547
CATEGORY_3,547
CATEGORY_4,547
MANUFACTURER,0
BRAND,0
BARCODE,0


All the other category columns of the products file are null where 'CATEGORY_1' column is 'Needs Review'. Let us look at the other columns:

In [65]:
products_cat1_NR = product_real[product_real['CATEGORY_1']=='Needs Review']

In [66]:
products_cat1_NR['BRAND'].value_counts()

,count
BRAND,
POLAR,145
COCA-COLA,139
FRITO-LAY,104
PEPSI,34
CHEETOS,27
SARGENTO,27
SNICKERS,23
COORS LIGHT,22
BUBLY SPARKLING WATER,21


In [67]:
products_cat1_NR['MANUFACTURER'].value_counts()

,count
MANUFACTURER,
PEPSICO,186
KEURIG DR PEPPER,145
THE COCA-COLA COMPANY,139
SARGENTO,27
MARS WRIGLEY,23
MOLSONCOORS,22
GALLO,3
GENERAL MILLS,2


It can be seen that most of the items are beverages and snacks.

Let us now check for the columns where 'CATEGORY_1' column is null

In [68]:
product_cat1_na = product_real[product_real['CATEGORY_1'].isna()]
product_cat1_na.shape

(111, 7)

In [69]:
product_cat1_na.isna().sum()

,0
CATEGORY_1,111
CATEGORY_2,111
CATEGORY_3,111
CATEGORY_4,111
MANUFACTURER,0
BRAND,0
BARCODE,0


It can be observed further that, all the other categories are null where 'CATEGORY_1' column is null.

Let us check which manufacturers and brands are mainly present whose categories are not known

In [70]:
product_cat1_na['BRAND'].value_counts()

,count
BRAND,
COCA-COLA,31
CHEETOS,27
COORS LIGHT,20
POLAR,15
SNICKERS,5
PEPSI,5
FRITO-LAY,3
BUBLY SPARKLING WATER,2
CHEERIOS,1


Another interesting observation here is that most of the brands whose categories are not filled belong to either beverages or snacks.

**Focus on the CATEGORY_2 column**


In [71]:
product_real.groupby(['CATEGORY_1','CATEGORY_2'])['CATEGORY_3'].nunique().sort_values(ascending=False)

CATEGORY_1         CATEGORY_2             
Health & Wellness  Skin Care                  16
                   Medicines & Treatments     14
                   Bath & Body                12
                   Hair Care                  11
Snacks             Dips & Salsa               10
                                              ..
                   Cookies                     0
Media              Books                       0
                   Music & Recordings          0
Office & School    Name Plates                 0
Vehicles & Parts   Vehicle Storage & Cargo     0
Name: CATEGORY_3, Length: 121, dtype: int64

In [72]:
product_cat2_na = product_real[product_real['CATEGORY_2'].isna()]
product_cat2_na.shape

(1422, 7)

In [73]:
product_cat2_na.isna().sum()

,0
CATEGORY_1,111
CATEGORY_2,1422
CATEGORY_3,1422
CATEGORY_4,1422
MANUFACTURER,2
BRAND,2
BARCODE,761


In [74]:
product_cat2_na['CATEGORY_1'].value_counts()

,count
CATEGORY_1,
Needs Review,547
Health & Wellness,378
Snacks,377
Beverages,2
Apparel & Accessories,2
Alcohol,2
Deli & Bakery,1
Pantry,1
Dairy,1


**Focus on CATEGORY_3 and CATEGORY_4 columns**

In [75]:
product_real.groupby('CATEGORY_1')['CATEGORY_3'].nunique().sort_values(ascending=False)

,CATEGORY_3
CATEGORY_1,
Health & Wellness,106
Snacks,74
Pantry,38
Home & Garden,22
Beverages,18
Apparel & Accessories,14
Alcohol,13
Sporting Goods,10
Dairy,8


In [76]:
product_cat3_na = product_real[product_real['CATEGORY_3'].isna()]
product_cat3_na.shape

(60563, 7)

In [77]:
product_cat3_na.isna().sum()

,0
CATEGORY_1,111
CATEGORY_2,1422
CATEGORY_3,60563
CATEGORY_4,60563
MANUFACTURER,20314
BRAND,20314
BARCODE,1849


In [78]:
product_cat3_na['CATEGORY_1'].value_counts()

,count
CATEGORY_1,
Snacks,41646
Health & Wellness,18084
Needs Review,547
Alcohol,62
Office & School,39
Beverages,20
Home & Garden,13
Apparel & Accessories,11
Animals & Pet Supplies,9


In [79]:
product_real.groupby('CATEGORY_1')['CATEGORY_4'].nunique().sort_values(ascending=False)

,CATEGORY_4
CATEGORY_1,
Health & Wellness,28
Alcohol,24
Home & Garden,17
Beverages,16
Pantry,16
Apparel & Accessories,6
Frozen,5
Dairy,4
Snacks,3


In [80]:
product_cat4_na = product_real[product_real['CATEGORY_4'].isna()]
product_cat4_na.shape

(778040, 7)

In [81]:
cat4_all_na = product_cat4_na.isna().sum()
cat4_all_na

,0
CATEGORY_1,111
CATEGORY_2,1422
CATEGORY_3,60563
CATEGORY_4,778040
MANUFACTURER,211108
BRAND,211106
BARCODE,3749


In [82]:
cat1_break = product_cat4_na['CATEGORY_1'].value_counts()
cat1_break

,count
CATEGORY_1,
Health & Wellness,460080
Snacks,314795
Pantry,808
Apparel & Accessories,767
Needs Review,547
Beverages,275
Dairy,212
Alcohol,97
Deli & Bakery,69


In [83]:
heal_well = cat1_break.iloc[0]/cat1_break.sum()
heal_well

0.5914164403178183

In [84]:
mis_bar_rat = cat4_all_na.iloc[6]
mis_bar_rat/product_real['BARCODE'].isna().sum()

0.9448084677419355

**Data Quality Issues** \
It can be noticed that 59.14% of the missing "CATEGORY_4" values are from the health and Awareness category. \
Secondly, 94.48% of the missing barcode values have missing category_4 values.

**Focus on the "BRAND" and "MANUFACTURER" columns**


In [85]:
product_real.groupby('CATEGORY_1').agg({'BRAND': 'nunique', 'MANUFACTURER': 'nunique'})

,BRAND,MANUFACTURER
CATEGORY_1,,
Alcohol,4,4
Animals & Pet Supplies,1,1
Apparel & Accessories,7,5
Arts & Entertainment,3,3
Baby & Toddler,2,2
Beauty,2,1
Beverages,5,4
Dairy,4,3
Deli & Bakery,3,3


In [86]:
product_brand_na = product_real[product_real['BRAND'].isna()]
product_brand_na.shape

(226472, 7)

In [87]:
product_brand_na.isna().sum()

,0
CATEGORY_1,0
CATEGORY_2,2
CATEGORY_3,20314
CATEGORY_4,211106
MANUFACTURER,226472
BRAND,226472
BARCODE,247


In [88]:
product_manu_na = product_real[product_real['MANUFACTURER'].isna()]
product_manu_na.shape

(226474, 7)

In [89]:
product_manu_na.isna().sum()

,0
CATEGORY_1,0
CATEGORY_2,2
CATEGORY_3,20314
CATEGORY_4,211108
MANUFACTURER,226474
BRAND,226472
BARCODE,247


In [90]:
product_brand_na['CATEGORY_1'].value_counts()

,count
CATEGORY_1,
Health & Wellness,128401
Snacks,98071


In [91]:
product_manu_na['CATEGORY_1'].value_counts()

,count
CATEGORY_1,
Health & Wellness,128403
Snacks,98071


All the missing brand and manufacturer details belong to two categories - Health & Wellness and Snacks.

**Focus on the BARCODE column**

In [92]:
product_real['barcode_is_12_digit'] = product_real['BARCODE'].map(lambda x: len(str(x)) == 12)
print(product_real['barcode_is_12_digit'].value_counts())

barcode_is_12_digit
True     768220
False     77275
Name: count, dtype: int64


<ipython-input-92-da4ed5253986>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_real['barcode_is_12_digit'] = product_real['BARCODE'].map(lambda x: len(str(x)) == 12)


In [93]:
false_barcodes = product_real[(product_real['barcode_is_12_digit'] == False) & (product_real['BARCODE'].notna())]
false_barcodes.shape

(73307, 8)

***Data Quality Issue*** \
There are 73307 non-null records (8.71% of all the non-null values) where the barcodes do not follow the 12 digit format.

In [94]:
#number of records where the barcode is missing.
product_real['BARCODE'].isna().sum()

3968

It can be noticed from the above results, almost 4000 rows are there where the barcode column is null. Let us analyze more.

In [95]:
product_bar_na = product_real[product_real['BARCODE'].isna()]
product_bar_na.isna().sum()

,0
CATEGORY_1,0
CATEGORY_2,761
CATEGORY_3,1849
CATEGORY_4,3749
MANUFACTURER,247
BRAND,247
BARCODE,3968
barcode_is_12_digit,0


In [96]:
#Grouping the records firstly according to category_1 followed by according to category_2 where the barcode is null.
brand_count = product_bar_na.groupby(['CATEGORY_1','CATEGORY_2'])['BRAND'].count().sort_values(ascending=False)
brand_count

CATEGORY_1             CATEGORY_2                              
Health & Wellness      Skin Care                                   378
                       Medicines & Treatments                      350
                       Bath & Body                                 325
                       Hair Care                                   298
Snacks                 Candy                                       257
                       Snack Bars                                  149
                       Chips                                       130
                       Crackers                                    105
Health & Wellness      Hair Removal                                 99
                       Oral Care                                    86
Snacks                 Dips & Salsa                                 86
                       Puffed Snacks                                85
                       Snack Cakes                                  64
Health & Wellness      Deodorant & Antiperspirant                   62
Snacks                 Fruit & Vegetable Snacks                     50
Health & Wellness      Eye Care                                     39
Snacks                 Pudding & Gelatin                            34
                       Cookies                                      33
                       Pretzels                                     32
Health & Wellness      First Aid                                    32
Snacks                 Nuts & Seeds                                 29
Health & Wellness      Topical Muscle & Joint Relief Treatments     27
                       Menstrual Care                               26
Snacks                 Dessert Toppings                             25
                       Snack Mixes                                  25
Alcohol                Wine                                         22
Snacks                 Jerky & Dried Meat                           17
Health & Wellness      Sexual Health                                16
                       Adult Incontinence                           15
Dairy                  Cheese                                        9
Health & Wellness      Foot Care                                     8
Snacks                 Trail Mix                                     6
                       Energy & Endurance                            6
Beverages              Carbonated Soft Drinks                        6
Restaurant             Beverages                                     5
Health & Wellness      Medical Supplies & Equipment                  5
Beverages              Water                                         4
Apparel & Accessories  Clothing                                      4
Health & Wellness      Ear Care                                      4
Alcohol                Beer                                          3
Pantry                 Cereal, Granola, & Toaster Pastries           2
Deli & Bakery          Prepared Meals                                2
Restaurant             À La Carte Item                               1
Snacks                 Variety Snack Packs                           1
Name: BRAND, dtype: int64

It can be observed that most of the missing barcodes are in the 'Health and Wellness' category with the highest in skincare followed by 'Snacks' Category with the highest in candy.

# Transactions Data Analysis

In [97]:
transact.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,015300014978,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,078742229751,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,783399746536,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,047900501183,1.00,


Let us rename the "ID" column to "USER_ID"

In [98]:
transact.rename(columns={'ID' : 'USER_ID'},inplace=True)

In [99]:
transact.shape

(50000, 8)

**Checking the datatypes**

In [100]:
transact.dtypes

,0
RECEIPT_ID,object
PURCHASE_DATE,object
SCAN_DATE,object
STORE_NAME,object
USER_ID,object
BARCODE,object
FINAL_QUANTITY,object
FINAL_SALE,object


**Checking for Duplicate Records**

In [101]:
transact_dupe = transact.duplicated().sum()
print("Total number of duplicate values in the transaction dataset:", transact_dupe)

Total number of duplicate values in the transaction dataset: 171


***Data Quality Issue*** \
There are 171 duplicate records.

In [102]:
#Removing the duplicate values
transact.drop_duplicates(inplace=True)

In [103]:
transact.describe(include='all')

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
count,49829,49829,49829,49829,49829,44094,49829,49829
unique,24440,89,24440,954,17694,11028,87,1435
top,0fb89572-c817-47e2-bd11-6f467baacbb2,2024-07-03,2024-06-25 17:51:10.487 Z,WALMART,64e62de5ca929250373e6cf5,078742223759,1.00,
freq,6,772,6,21249,22,181,35536,12486


***Data Quality Issues*** \
There are many issues that can be observed here: \
1) Though there are 49,829 recorded observations, very few values are unique in each column. A deeper dive into the matter is needed. Especially the scan_date column, because it contains even the time-stamp and if there 49,829 recorded observations and only 24440 are unique values, there is a high chance that a lot of scanned items were recorded more than once! \
2) Just the barcode column has missing values with a lot of recurring values. This might be an issue when merging the tables for analyses. \
3) The data in 'FINAL_QUANTITY' column and the 'FINAL_SALE' column seems ambiguous. Further analysis is needed. Additionally, both the columns should be in integer format. So, we will convert them into a numeric datatype after the analysis into the presence of spaces.



In [104]:
transact.nunique()

,0
RECEIPT_ID,24440
PURCHASE_DATE,89
SCAN_DATE,24440
STORE_NAME,954
USER_ID,17694
BARCODE,11028
FINAL_QUANTITY,87
FINAL_SALE,1435


**Converting PURCHASE_DATE and SCAN_DATE into date-time objects**

In [105]:
transact['PURCHASE_DATE'] = pd.to_datetime(transact['PURCHASE_DATE'])
transact['SCAN_DATE'] = pd.to_datetime(transact['SCAN_DATE'])


transact['SCAN_DATE'] = transact['SCAN_DATE'].dt.tz_convert('UTC')

transact.dtypes

,0
RECEIPT_ID,object
PURCHASE_DATE,datetime64[ns]
SCAN_DATE,"datetime64[ns, UTC]"
STORE_NAME,object
USER_ID,object
BARCODE,object
FINAL_QUANTITY,object
FINAL_SALE,object


In [106]:
transact.describe(include='all')

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
count,49829,49829,49829,49829,49829,44094,49829,49829
unique,24440,NaN,NaN,954,17694,11028,87,1435
top,0fb89572-c817-47e2-bd11-6f467baacbb2,NaN,NaN,WALMART,64e62de5ca929250373e6cf5,078742223759,1.00,
freq,6,NaN,NaN,21249,22,181,35536,12486
mean,NaN,2024-07-24 08:05:52.565775104,2024-07-26 22:53:49.655994368+00:00,NaN,NaN,NaN,NaN,NaN
min,NaN,2024-06-12 00:00:00,2024-06-12 06:36:34.910000+00:00,NaN,NaN,NaN,NaN,NaN
25%,NaN,2024-07-03 00:00:00,2024-07-05 10:44:40.377999872+00:00,NaN,NaN,NaN,NaN,NaN
50%,NaN,2024-07-23 00:00:00,2024-07-26 10:57:39.848999936+00:00,NaN,NaN,NaN,NaN,NaN
75%,NaN,2024-08-15 00:00:00,2024-08-17 15:39:25.552999936+00:00,NaN,NaN,NaN,NaN,NaN
max,NaN,2024-09-08 00:00:00,2024-09-08 23:07:19.836000+00:00,NaN,NaN,NaN,NaN,NaN


Let us check for the presence of data structures in the columns.

In [107]:
def check_column_datastructures(column):
    return any(isinstance(x, (list, dict, np.ndarray)) for x in column)

# Applying the function to each column
columns_with_structures = {col: check_column_datastructures(transact[col]) for col in transact.columns}

# Displaying which columns contain arrays, lists, or dictionaries
print("Columns containing arrays, lists, or dictionaries:")
for col, contains_structure in columns_with_structures.items():
    if contains_structure:
        print(f"{col} contains complex structures")

Columns containing arrays, lists, or dictionaries:


**Checking for Null Values**

In [108]:
transact.isna().sum()

,0
RECEIPT_ID,0
PURCHASE_DATE,0
SCAN_DATE,0
STORE_NAME,0
USER_ID,0
BARCODE,5735
FINAL_QUANTITY,0
FINAL_SALE,0


**Checking if the purchase date is before the scanned date for all the records.**


In [109]:
# Comparing Purchasing dates and Scanning dates and storing it in a new variable
transact['PD<SD'] = transact['PURCHASE_DATE'] <= transact['SCAN_DATE'].dt.date

transact['PD<SD'].value_counts()

,count
PD<SD,
True,49735
False,94


***Data Quality Issue*** \
94 records show that the scanned date is before than the purchased date. This means that the users scanned the receipt before they made a purchase.


Let us focus on three columns in particular and analyze them with respect to the other columns. The three columns to focus on are  - 'BARCODE', 'FINAL_SALE' and 'FINAL_QUANTITY'.

**Focus on 'FINAL_SALE' and 'FINAL_QUANTITY' columns**

In [110]:
transact['FINAL_QUANTITY'].value_counts()

,count
FINAL_QUANTITY,
1.00,35536
zero,12491
2.00,1285
3.00,184
4.00,139
...,...
6.22,1
1.22,1
1.23,1


In [111]:
transact['FINAL_SALE'].value_counts()

,count
FINAL_SALE,
,12486
1.25,1313
1.00,732
2.99,587
1.99,581
...,...
16.47,1
10.92,1
61.30,1


In [112]:
transact_sale_blank = transact[transact['FINAL_SALE']==' ']
transact_sale_blank.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,PD<SD
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539000+00:00,WALMART,63b73a7f3d310dceeabd4758,015300014978,1.00,,True
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813000+00:00,WALMART,60842f207ac8b7729e472020,078742229751,1.00,,True
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549000+00:00,RANDALLS,6193231ae9b3d75037b0f928,047900501183,1.00,,True
6,000550b2-1480-4c07-950f-ff601f242152,2024-07-06,2024-07-06 19:27:48.586000+00:00,WALMART,5f850bc9cf9431165f3ac175,049200905548,1.00,,True
8,000e1d35-15e5-46c6-b6b3-33653ed3d27e,2024-08-13,2024-08-13 18:21:07.931000+00:00,WALMART,61a6d926f998e47aad33db66,052000011227,1.00,,True


In [113]:
transact_sale_blank.shape

(12486, 9)

In [114]:
transact_sale_blank['FINAL_QUANTITY'].nunique()

41

In [115]:
transact_sale_blank['FINAL_QUANTITY'].value_counts()

,count
FINAL_QUANTITY,
1.00,11900
2.00,427
3.00,62
4.00,43
6.00,8
5.00,5
7.00,3
1.40,2
12.00,2


In [116]:
#Creating a dataframe which doesnt have any blank "FINAL_SALE" rows
transact_sale_NB = transact[transact['FINAL_SALE'] != ' ']


To understand the blanks in 'final_sale' column, let us see if there are any common barcodes in the dataframe where 'final_sale' has only blanks and the dataframe which consists the rest of the data.

In [117]:
#Finding common barcodes in two dataframes, one with the blanks and one without the blanks.
common_barcodes = set(transact_sale_NB['BARCODE']).intersection(set(transact_sale_blank['BARCODE']))
combar_num = len(common_barcodes)
combar_num


6805

It can be seen that 6805 barcodes are common, it means that the sale value would be the same for all these items. So we can replace the blank space with the corresponding sales value where the barcode is identical.

Let us now check for the number of unique barcode values in the dataframe where the sale_value is blank.

In [118]:
#Creating an array with unique barcode values where the FINAL_SALE column is blank
x = transact_sale_blank['BARCODE'].unique()
len(x)

6805

This is a surprising answer, this is equal to the common barcode values! This proves that: \
1) There are no additional unique barcodes in the dataframe with a blank "FINAL_SALE" column that are not present in the dataframe with the rest of the values. \
2) All the 12486 rows with the blank "FINAL_SALE" columns can be filled with the corresponding "FINAL_SALE" values where the barcodes match.

Let us now crosscheck our findings - Let us check if there there are any matches between the common barcodes and unique barcodes of the dataframe with the blank sales column

In [119]:
blank_bar_unique = set(transact_sale_blank['BARCODE'].unique())
# Finding additional unique barcodes that may be present in the dataset with blank 'FINAL_SALE' values.
missing_barcode =  blank_bar_unique - common_barcodes
missing_barcode


set()

It is an empty set! So this proves the first point in the above conclusion!

Before filling the blank spaces with their corresponding "FINAL_SALE" values, let us understand why this issue arose in the first place by looking at all the instances when a particular barcode got recorded into the transaction records.

In [120]:
#Considering a random barcode which has blank value in "FINAL_SALE"
filter_bycode1 = transact[transact['BARCODE'] == '511111045496']
filter_bycode1.shape


(20, 9)

In [121]:
filter_bycode1.groupby(['RECEIPT_ID','USER_ID','STORE_NAME','SCAN_DATE','PURCHASE_DATE','FINAL_SALE','FINAL_QUANTITY'])['BARCODE'].apply(list)

RECEIPT_ID                            USER_ID                   STORE_NAME            SCAN_DATE                         PURCHASE_DATE  FINAL_SALE  FINAL_QUANTITY
1542749e-101f-430d-8b33-fd8a200fe1de  5ddefc35ce77bc78062d42fb  PUBLIX                2024-07-21 15:58:37.476000+00:00  2024-07-20                 1.00              [511111045496]
                                                                                                                                       0.99        1.00              [511111045496]
17f0820c-c480-4482-80e1-796670646674  5cad608ae03e4b18c625d2da  WINN-DIXIE            2024-06-23 12:29:25.252000+00:00  2024-06-20     4.99        1.00              [511111045496]
                                                                                                                                                   zero              [511111045496]
294a2785-22be-4a4f-b8a9-f387991ad0fa  5ecb23aea0134313d2d8a3dd  KROGER                2024-06-23 08:43:56.378000+00:00  2024-06-17                 1.00              [511111045496]
                                                                                                                                       2.50        1.00              [511111045496]
486c09c9-c5ce-4d4f-b004-024344662d13  666f1d84465f309038ab3a6c  PRICE CHOPPER         2024-09-02 13:54:37.016000+00:00  2024-09-02     1.00        1.00              [511111045496]
                                                                                                                                                   zero              [511111045496]
67e06a61-0eb4-4138-bdf0-c6e46c9a4526  63dc6c96dcb50fbd3083edb1  SHOP RITE             2024-06-27 12:31:18.008000+00:00  2024-06-20     1.27        1.00              [511111045496]
                                                                                                                                                   zero              [511111045496]
82fb7d5d-3bcd-4aab-a079-45447bf4b4a5  6621f37cc41e9f27acd82170  STOP & SHOP           2024-06-16 18:20:19.585000+00:00  2024-06-15                 1.00              [511111045496]
                                                                                                                                       0.57        1.00              [511111045496]
89497076-6f2b-45cb-b879-6a8fa6dc7700  66259438c41e9f27acd89e2c  BERKOT'S SUPER FOODS  2024-07-24 17:07:27.389000+00:00  2024-07-24                 1.00              [511111045496]
                                                                                                                                       4.00        1.00              [511111045496]
8d9897c0-8a25-46f1-8a2b-d53d6d6da64e  6248d2b5834ddd30c4f192e1  STOP & SHOP           2024-08-31 11:24:07.903000+00:00  2024-08-17                 1.00              [511111045496]
                                                                                                                                       3.34        1.00              [511111045496]
e4117047-688b-4018-a722-7f8f5fef65b4  5e10fb7cf3aee111fba9f939  KING SOOPERS          2024-07-29 20:10:38.893000+00:00  2024-07-29                 1.00              [511111045496]
                                                                                                                                       2.49        1.00              [511111045496]
f8de4eb9-1dfd-44eb-b4bc-6133b5466f27  6650b17c7c0469953bf9a1c6  PRICE CHOPPER         2024-09-02 17:37:01.192000+00:00  2024-08-23     1.00        1.00              [511111045496]
                                                                                                                                                   zero              [511111045496]
Name: BARCODE, dtype: object

In [122]:
#Considering another barcode which is present in the dataframe where "FINAL_SALE" is blank.
filter_bycode2 = transact[transact['BARCODE'] == '048001353664']
filter_bycode2.shape


(6, 9)

In [123]:
filter_bycode2.groupby(['RECEIPT_ID','USER_ID','STORE_NAME','SCAN_DATE','PURCHASE_DATE','FINAL_SALE','FINAL_QUANTITY'])['BARCODE'].apply(list)

RECEIPT_ID                            USER_ID                   STORE_NAME          SCAN_DATE                         PURCHASE_DATE  FINAL_SALE  FINAL_QUANTITY
51664568-80e1-4f19-9382-8c08849581ba  608310dbeb15d564f519e93e  WALMART             2024-08-17 18:04:28.207000+00:00  2024-08-16     4.38        1.00              [048001353664]
                                                                                                                                                 zero              [048001353664]
5726fd13-9319-4f47-9f37-f9820954bebc  624eefc9f0c39f0743e609fb  PETE'S MARKET       2024-06-23 22:21:33.516000+00:00  2024-06-21                 1.00              [048001353664]
                                                                                                                                     4.99        1.00              [048001353664]
fe0780d1-2d02-4822-8f12-7056b1814f17  5d197f9dd08976510c49d0e6  MAIN STREET MARKET  2024-08-11 17:52:18.523000+00:00  2024-08-09                 276.00            [048001353664]
                                                                                                                                     5.89        276.00            [048001353664]
Name: BARCODE, dtype: object

In [124]:
#Considering another random barcode which is present in the dataframe where "FINAL_SALE" is blank.
filter_bycode3 = transact[transact['BARCODE'] == '311111224057']
filter_bycode3.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,PD<SD
36,006490a1-65b3-4ec7-8130-08767846432a,2024-09-02,2024-09-02 12:18:56.915000+00:00,PUBLIX,617cb0e1e5e5187841b701dc,311111224057,1.00,,True
171,01cf54de-0f0b-4a89-aa35-227ba103ed59,2024-08-10,2024-08-15 07:58:59.635000+00:00,PLAVERS CAFE LIDA,62ec127f4e73e2db30eaadae,311111224057,zero,6.00,True
880,08e5e4a4-f72e-44ed-984e-28d66d565410,2024-08-09,2024-08-11 18:18:03.811000+00:00,ALDI,60a31588671b6805df74cd1e,311111224057,1.00,,True
943,097724fe-0b26-48b9-acd2-706f566cb5a7,2024-07-14,2024-07-14 08:37:49.898000+00:00,RALPH'S CAQUAS,637e59e9d6f2a49c49941f38,311111224057,zero,1.75,True
997,0a105723-7aab-46a1-9016-97177a5a7314,2024-09-05,2024-09-05 22:13:41.844000+00:00,SUPERMAX,635d382e68d1ed9fd3b30e09,311111224057,zero,1.42,True


In [125]:
filter_bycode3.shape

(149, 9)

In [126]:
filter_bycode3['STORE_NAME'].value_counts()

,count
STORE_NAME,
KROGER,10
ALDI,10
SHOP RITE,10
PUBLIX,6
SHELL,4
MARTIN'S SUPERMARKET,4
TARGET,4
WALMART,4
PUEBLO,3


In [127]:
#Checking the FINAL_QUANTITY and FINAL_COLUMNS where the barcode is identical in 3 instances.
filter_bycode3[filter_bycode3['STORE_NAME']=='PUEBLO']

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,PD<SD
11814,7a09e2bc-53e0-411e-a72f-9ff3b636bf46,2024-08-25,2024-08-25 14:52:08.512000+00:00,PUEBLO,665b575ce04f743a096ca7ea,311111224057,1.00,,True
11815,7a09e2bc-53e0-411e-a72f-9ff3b636bf46,2024-08-25,2024-08-25 14:52:08.512000+00:00,PUEBLO,665b575ce04f743a096ca7ea,311111224057,zero,0.79,True
42493,7a09e2bc-53e0-411e-a72f-9ff3b636bf46,2024-08-25,2024-08-25 14:52:08.512000+00:00,PUEBLO,665b575ce04f743a096ca7ea,311111224057,1.00,0.79,True


In [128]:
filter_bycode3.groupby(['RECEIPT_ID','USER_ID','STORE_NAME','SCAN_DATE','PURCHASE_DATE',
                        'FINAL_SALE','FINAL_QUANTITY'])['BARCODE'].apply(list).head(20)

RECEIPT_ID                            USER_ID                   STORE_NAME                          SCAN_DATE                         PURCHASE_DATE  FINAL_SALE  FINAL_QUANTITY
006490a1-65b3-4ec7-8130-08767846432a  617cb0e1e5e5187841b701dc  PUBLIX                              2024-09-02 12:18:56.915000+00:00  2024-09-02                 1.00              [311111224057]
                                                                                                                                                     7.99        1.00              [311111224057]
01cf54de-0f0b-4a89-aa35-227ba103ed59  62ec127f4e73e2db30eaadae  PLAVERS CAFE LIDA                   2024-08-15 07:58:59.635000+00:00  2024-08-10     6.00        2.00              [311111224057]
                                                                                                                                                                 zero              [311111224057]
08e5e4a4-f72e-44ed-984e-28d66d565410  60a31588671b6805df74cd1e  ALDI                                2024-08-11 18:18:03.811000+00:00  2024-08-09                 1.00              [311111224057]
                                                                                                                                                     2.68        1.00              [311111224057]
097724fe-0b26-48b9-acd2-706f566cb5a7  637e59e9d6f2a49c49941f38  RALPH'S CAQUAS                      2024-07-14 08:37:49.898000+00:00  2024-07-14     1.75        1.00              [311111224057]
                                                                                                                                                                 zero              [311111224057]
0a105723-7aab-46a1-9016-97177a5a7314  635d382e68d1ed9fd3b30e09  SUPERMAX                            2024-09-05 22:13:41.844000+00:00  2024-09-05     1.42        1.00              [311111224057]
                                                                                                                                                                 zero              [311111224057]
0ab1d88e-35b0-458e-b4cb-aed043360a0e  5e46c512a22eaf13e49fd96f  FOOD LION                           2024-07-15 09:16:27.326000+00:00  2024-07-12                 1.00              [311111224057]
                                                                                                                                                     13.98       1.00              [311111224057]
0dbeebbd-7e0d-4ea1-82a3-02c68d224bd1  5b0018e7e4b0dbe82b980143  ANAZEH SANDS BILLIARDS              2024-08-13 18:05:30.322000+00:00  2024-08-11     2.83        1.00              [311111224057]
                                                                                                                                                                 zero              [311111224057]
0f8cfb7d-726a-415a-ab8c-249b627e8069  6001f21496277812d5264b99  ZOOB ZIB THAI AUTHENTIC NOODLE BAR  2024-07-12 13:17:14.162000+00:00  2024-07-12     2.50        1.00              [311111224057]
                                                                                                                                                                 zero              [311111224057]
1610139d-c396-4ed9-8e64-b05baec8118f  6670c1c6465f309038ab7fb2  KISMET BIALYS                       2024-07-27 17:00:13.149000+00:00  2024-07-27     3.00        1.00              [311111224057]
                                                                                                                                                                 zero              [311111224057]
1706ebb2-f3c3-4b47-b709-a4ee8f896a95  5dcb779ce8f21112aa365ca9  KROGER                              2024-07-24 06:18:16.699000+00:00  2024-07-23                 1.00              [311111224057]
                                                                                                                                          

Let us see if this is the case with every transaction in the dataset. Did every transaction get recorded more than once for the same time? Let us consider the Barcode and check if any value in the barcode has appeared just once. This would prove that some transactions happened only once.

In [129]:
#Checking if each unique value in the "BARCODE" column has appeared only once
unique_barcode = transact['BARCODE'].value_counts()
single_barcode_occurrence = unique_barcode[unique_barcode == 1].index.tolist()
print("Values that appear only once in BARCODE column:", single_barcode_occurrence)



Values that appear only once in BARCODE column: []


***Major Data Quality Issue*** \
Our earlier decision of replacing the blank 'FINAL_SALE' values with their corresponding counterpart would have proved highly detrimental for the analysis. On further inspection, it was found that there is a presence of duplicate entries!\
It seems that every item got scanned more than once at the same time and date for every user! An interesting observation here is, wherever the sale value and the quantity are recorded correctly for an item at the particular time, then the rest of the duplicate records that got recorded at the same scanned time have the following changes: \
 Either the sale value is left blank or the sale quantity is mentioned as "zero"


**Solution to correct this data discrepancy to go forward with the asked business insights** \
The rows where "zero" is written in the "FINAL_QUANTITY" and the rows where there is a blank "FINAL_SALE" column will be removed.


In [130]:
transact_filtered = transact[~((transact['FINAL_QUANTITY'] == 'zero') | (transact['FINAL_SALE'] == ' '))]
transact_filtered.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,PD<SD
25000,7b3ec72d-9d30-40b8-b185-0bfb638942a9,2024-08-20,2024-08-20 11:17:29.633000+00:00,DOLLAR GENERAL STORE,60fc1e6deb7585430ff52ee7,745527114884,1.00,1.65,True
25001,04869b68-29e3-4e8d-9bdb-950046fc3473,2024-08-05,2024-08-09 16:06:00.570000+00:00,DOLLAR GENERAL STORE,654cf234a225ea102b81072e,745527114884,1.00,1.65,True
25002,f1a96308-24a5-46a8-8d8c-285cf9dce1ba,2024-09-03,2024-09-03 11:28:25.264000+00:00,WALMART,63c1cb6d3d310dceeac55487,037000828761,1.00,28.22,True
25003,7ee1798e-fd2e-4278-838b-f417fdcafe08,2024-08-30,2024-09-04 12:53:31.478000+00:00,DOLLAR GENERAL STORE,65c29b137050d0a6206cd24f,012000504051,1.00,5.25,True
25004,21feab39-49f2-42e9-ae69-10371e2fc0a9,2024-08-23,2024-08-27 10:45:00.125000+00:00,TARGET,61a58ac49c135b462ccddd1c,024000393429,1.00,2.59,True


In [131]:
transact_filtered.shape

(24852, 9)

Let us check investigate more into our changes

In [132]:
transact_filtered.describe(include='all')

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,PD<SD
count,24852,24852,24852,24852,24852,21996,24852,24852,24852
unique,24440,NaN,NaN,954,17694,11028,86,1434,2
top,dd03ea1b-0fae-4bcf-bb55-d7e36eaa75b5,NaN,NaN,WALMART,64e62de5ca929250373e6cf5,078742223759,1.00,1.25,True
freq,3,NaN,NaN,10595,11,90,23636,880,24805
mean,NaN,2024-07-24 07:23:19.323997952,2024-07-26 22:11:20.974289408+00:00,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,2024-06-12 00:00:00,2024-06-12 06:36:34.910000+00:00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2024-07-03 00:00:00,2024-07-05 10:18:02.056999936+00:00,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,2024-07-23 00:00:00,2024-07-26 10:04:11.527500032+00:00,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,2024-08-15 00:00:00,2024-08-17 14:49:51.241250048+00:00,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,2024-09-08 00:00:00,2024-09-08 23:07:19.836000+00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
transact_filtered.isna().sum()

,0
RECEIPT_ID,0
PURCHASE_DATE,0
SCAN_DATE,0
STORE_NAME,0
USER_ID,0
BARCODE,2856
FINAL_QUANTITY,0
FINAL_SALE,0
PD<SD,0


Let us now convert the "FINAL_SALE" and "FINAL_QUANTITY" columns into numeric

In [134]:
transact_filtered.loc[:, ['FINAL_SALE', 'FINAL_QUANTITY']] = transact_filtered[['FINAL_SALE', 'FINAL_QUANTITY']].apply(pd.to_numeric,
                                                                                                                       errors='coerce')

In [135]:
transact_filtered.dtypes

,0
RECEIPT_ID,object
PURCHASE_DATE,datetime64[ns]
SCAN_DATE,"datetime64[ns, UTC]"
STORE_NAME,object
USER_ID,object
BARCODE,object
FINAL_QUANTITY,object
FINAL_SALE,object
PD<SD,bool


Let us check for the legitimacy of the barcodes

In [136]:
transact_filtered.loc[:,'barcode_is_12_digit'] = transact_filtered['BARCODE'].map(lambda x: len(str(x)) == 12)
print(transact_filtered.loc[:,'barcode_is_12_digit'].value_counts())


barcode_is_12_digit
True     21933
False     2919
Name: count, dtype: int64


<ipython-input-136-bb2598c95050>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transact_filtered.loc[:,'barcode_is_12_digit'] = transact_filtered['BARCODE'].map(lambda x: len(str(x)) == 12)


In [137]:
invalid_transact_barcodes = transact_filtered[(transact_filtered['BARCODE'].notna()) & (transact_filtered['barcode_is_12_digit']==False)]
invalid_transact_barcodes.shape

(63, 10)

***Data Quality Issue*** \
There are 2856 null barcode values and an additional 63 invalid barcodes in the transactions database. This might be because of the application not scanning the barcodes properly or because of an improper use by the customer. Let us see for the common invalid barcodes in both products and transactions. \

In [138]:
common_invalid_barcodes = pd.merge(invalid_transact_barcodes[invalid_transact_barcodes['BARCODE'].notna()],
                                   false_barcodes[false_barcodes['BARCODE'].notna()], on='BARCODE', how='inner')
common_invalid_barcodes.shape

(56, 17)

There are 56 invalid barcodes which are common between both transactions and products.

Let us go through the filtered dataframe to check for any inconsistensies before finalizing the transactions dataset.

In [139]:
#Checking for receipt_id uniqueness
transact_filtered['RECEIPT_ID'].value_counts()

,count
RECEIPT_ID,
dd03ea1b-0fae-4bcf-bb55-d7e36eaa75b5,3
79151f8d-0b75-48e2-8bb4-2591bc8c9ca2,3
d6a313ee-1aa3-4acb-a90d-f0d962ae7b8c,3
2acd7e8d-37df-4e51-8ee5-9a9c8c1d9711,3
98d68d5d-71f1-4528-a83d-cdf6d308c79b,3
...,...
f2e87c6f-c9ef-4615-9ad2-a1b832f8f69e,1
ddab8fa2-159a-403d-b96c-f51f13ee4325,1
5640be8e-e8aa-4b5b-bb75-b8a07dd230ef,1


There are still some identical receipt_ids. We have to dig deeper to get more understanding.

In [140]:
#Grouping the dataset by receipt_id, scan_date and many others and listing out the barcodes
transact_diff = transact_filtered.groupby(['RECEIPT_ID','SCAN_DATE','USER_ID','STORE_NAME'])['BARCODE'].apply(list)
transact_diff.head()

,,,,BARCODE
RECEIPT_ID,SCAN_DATE,USER_ID,STORE_NAME,
0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21 14:19:06.539000+00:00,63b73a7f3d310dceeabd4758,WALMART,[015300014978]
0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20 09:50:24.206000+00:00,62c08877baa38d1a1f6c211a,ALDI,[nan]
00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-19 15:38:56.813000+00:00,60842f207ac8b7729e472020,WALMART,[078742229751]
000239aa-3478-453d-801e-66a82e39c8af,2024-06-19 11:03:37.468000+00:00,63fcd7cea4f8442c3386b589,FOOD LION,[783399746536]
00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-05 15:56:43.549000+00:00,6193231ae9b3d75037b0f928,RANDALLS,[047900501183]


In [141]:
# Filtering out groups with 2 or more barcode values
multi_barcode = transact_diff[transact_diff.apply(len) >= 2 ]
#Filtering out lists where not all barcode values are unique
multi_unique_barcodes = multi_barcode[multi_barcode.apply(lambda x: len(set(x)) != len(x))]
multi_unique_barcodes

,,,,BARCODE
RECEIPT_ID,SCAN_DATE,USER_ID,STORE_NAME,
00bf741b-24d6-4064-a267-f87748bb5aa9,2024-07-09 17:37:14.702000+00:00,62676081ed0f8765fb700454,ALDI,"[nan, nan]"
03eb6f94-0088-47ae-a839-4dfa585d7044,2024-07-13 08:57:06.269000+00:00,5b79b541719e5454ef4d2f01,ALDI,"[nan, nan]"
07d7195f-068e-4b8d-8f65-6ced98991b1f,2024-08-28 18:35:29.370000+00:00,639a7d5c4b2ebdb1fed8386e,ALDI,"[nan, nan]"
179a102f-41eb-4220-80cf-800439050b6a,2024-07-03 09:40:59.730000+00:00,64b5b0bc03525c5d2a305241,ALDI,"[nan, nan]"
19f725b1-e708-4060-b159-acc7674e5405,2024-09-03 09:47:23.950000+00:00,619c29ca06570b5913a151fb,SHOP RITE,"[311111921628, 311111921628]"
1d0bb09b-cc9b-40e7-9887-97736f6edf08,2024-07-31 18:32:35.734000+00:00,63ba250039c79dcbdd4fbc6c,COSTCO,"[nan, nan]"
1dad82ac-3e03-4e2a-98ae-3140335574f1,2024-09-04 20:02:05.951000+00:00,65f4782996ce12ad907a0d12,ALDI,"[nan, nan]"
20096213-a4a6-439b-9f2c-27acf385af72,2024-07-04 22:32:59.661000+00:00,64c017269b7ac0499fa579b4,WALGREENS,"[nan, nan]"
20ce57e4-5b53-46e9-b014-3bbf16fa96c0,2024-07-01 10:44:10.996000+00:00,60fd81bad418d04106e06bd0,WALGREENS,"[nan, nan]"


In [142]:
#Viewing some records where the recipt_id and the barcodes are identitical
receipt_view = transact_filtered[(transact_filtered['RECEIPT_ID']=='19f725b1-e708-4060-b159-acc7674e5405')|
                         (transact_filtered['RECEIPT_ID']=='431fe612-ed55-470e-939c-043ad31f33f3')]
receipt_view

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,PD<SD,barcode_is_12_digit
31668,19f725b1-e708-4060-b159-acc7674e5405,2024-08-23,2024-09-03 09:47:23.950000+00:00,SHOP RITE,619c29ca06570b5913a151fb,311111921628,1.0,2.99,True,True
31675,19f725b1-e708-4060-b159-acc7674e5405,2024-08-23,2024-09-03 09:47:23.950000+00:00,SHOP RITE,619c29ca06570b5913a151fb,311111921628,1.0,3.96,True,True
43005,431fe612-ed55-470e-939c-043ad31f33f3,2024-09-07,2024-09-07 16:39:01.409000+00:00,DOLLAR GENERAL STORE,5e038cebcb322c11de193bb7,012000504051,1.0,6.25,True,True
43006,431fe612-ed55-470e-939c-043ad31f33f3,2024-09-07,2024-09-07 16:39:01.409000+00:00,DOLLAR GENERAL STORE,5e038cebcb322c11de193bb7,012000504051,1.0,5.25,True,True
43007,431fe612-ed55-470e-939c-043ad31f33f3,2024-09-07,2024-09-07 16:39:01.409000+00:00,DOLLAR GENERAL STORE,5e038cebcb322c11de193bb7,012000504051,1.0,5.26,True,True


Let us see what these items are based on  the barcode

In [143]:
#Extracting the product details based on the barcode obtained from the above dataset
product_view = product_real[(product_real['BARCODE']=='311111921628')|
                         (product_real['BARCODE']=='012000504051')]
product_view

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit
821428,Beverages,Carbonated Soft Drinks,Cola,Diet Cola,PEPSICO,PEPSI,311111921628,True
827515,Beverages,Carbonated Soft Drinks,Cola,Regular Cola,PEPSICO,PEPSI,012000504051,True


In [144]:
product_view.shape

(2, 8)

***Potential Data Quality Issue***

It seems that there are identical barcodes multiple times on the same receipt number on the same date and time. The only difference is that, the "FINAL_SALE" values are different though the "FINAL_QUANTITY" is the same. \
First theory - It might be because a user bought two or more identical products but somehow the quantity got entered wrong. \
Second Theory - Another hypothesis is that the product might have been on an offer - something like, buy 1 for 5 dollars and get the second pack for 2.5 dollars.   
The second one makes more sense.\
But I would flag this as a data quality issue to understand it better.     

**Final Datasets after filtering**

In [145]:
users.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54+00:00,2000-08-11 00:00:00+00:00,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55+00:00,2001-09-24 04:00:00+00:00,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18+00:00,1994-10-28 00:00:00+00:00,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22+00:00,NaT,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50+00:00,1972-03-19 00:00:00+00:00,PA,en,female


In [146]:
users_final = users

In [147]:
product_real.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,796494407820,True
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,023278011028,True
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,461817824225,True
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,035000466815,True
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,0806810850459,False


In [148]:
product_final = product_real.drop(columns='barcode_is_12_digit',axis=1)
product_final.head()


,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,796494407820
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,023278011028
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,461817824225
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,035000466815
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,0806810850459


In [149]:
transact_filtered.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,PD<SD,barcode_is_12_digit
25000,7b3ec72d-9d30-40b8-b185-0bfb638942a9,2024-08-20,2024-08-20 11:17:29.633000+00:00,DOLLAR GENERAL STORE,60fc1e6deb7585430ff52ee7,745527114884,1.0,1.65,True,True
25001,04869b68-29e3-4e8d-9bdb-950046fc3473,2024-08-05,2024-08-09 16:06:00.570000+00:00,DOLLAR GENERAL STORE,654cf234a225ea102b81072e,745527114884,1.0,1.65,True,True
25002,f1a96308-24a5-46a8-8d8c-285cf9dce1ba,2024-09-03,2024-09-03 11:28:25.264000+00:00,WALMART,63c1cb6d3d310dceeac55487,037000828761,1.0,28.22,True,True
25003,7ee1798e-fd2e-4278-838b-f417fdcafe08,2024-08-30,2024-09-04 12:53:31.478000+00:00,DOLLAR GENERAL STORE,65c29b137050d0a6206cd24f,012000504051,1.0,5.25,True,True
25004,21feab39-49f2-42e9-ae69-10371e2fc0a9,2024-08-23,2024-08-27 10:45:00.125000+00:00,TARGET,61a58ac49c135b462ccddd1c,024000393429,1.0,2.59,True,True


In [150]:
transact_final = transact_filtered.drop(columns=['barcode_is_12_digit','PD<SD'],axis=1)
transact_final.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
25000,7b3ec72d-9d30-40b8-b185-0bfb638942a9,2024-08-20,2024-08-20 11:17:29.633000+00:00,DOLLAR GENERAL STORE,60fc1e6deb7585430ff52ee7,745527114884,1.0,1.65
25001,04869b68-29e3-4e8d-9bdb-950046fc3473,2024-08-05,2024-08-09 16:06:00.570000+00:00,DOLLAR GENERAL STORE,654cf234a225ea102b81072e,745527114884,1.0,1.65
25002,f1a96308-24a5-46a8-8d8c-285cf9dce1ba,2024-09-03,2024-09-03 11:28:25.264000+00:00,WALMART,63c1cb6d3d310dceeac55487,037000828761,1.0,28.22
25003,7ee1798e-fd2e-4278-838b-f417fdcafe08,2024-08-30,2024-09-04 12:53:31.478000+00:00,DOLLAR GENERAL STORE,65c29b137050d0a6206cd24f,012000504051,1.0,5.25
25004,21feab39-49f2-42e9-ae69-10371e2fc0a9,2024-08-23,2024-08-27 10:45:00.125000+00:00,TARGET,61a58ac49c135b462ccddd1c,024000393429,1.0,2.59


# Data Analysis for the SQL Queries of Section 2

**Let us focus on the dips and salsa category from the products table**


The question is about finding the top brand in the 'Dips & Salsa' category

In [151]:
# Looking at the unique values of the category_2 in the products table
product_real['CATEGORY_2'].unique()

array(['Sexual Health', 'Puffed Snacks', 'Hair Care', 'Oral Care',
       'Medicines & Treatments', 'Deodorant & Antiperspirant',
       'Snack Bars', nan, 'Bath & Body', 'Nuts & Seeds', 'Candy',
       'Cookies', 'Variety Snack Packs', 'Hair Removal',
       'Medical Supplies & Equipment', 'Chips', 'Snack Cakes',
       'Skin Care', 'Dessert Toppings', 'Eye Care',
       'Fruit & Vegetable Snacks', 'Snack Mixes', 'Crackers',
       'Jerky & Dried Meat', 'Topical Muscle & Joint Relief Treatments',
       'Foot Care', 'First Aid', 'Ear Care', 'Menstrual Care', 'Pretzels',
       'Trail Mix', 'Dips & Salsa', 'Adult Incontinence', 'Water',
       'Cereal, Granola, & Toaster Pastries', 'Wine', 'Pudding & Gelatin',
       'Clothing', 'Carbonated Soft Drinks', 'Energy & Endurance',
       'Beverages', 'Cheese', 'Kitchen & Dining', 'Energy Drinks',
       'Packaged Vegetables', 'Household Cleaning Supplies',
       'Business & Home Security',
       'Hard Seltzers, Sodas, Waters, Lemonades & 

In [152]:
# Considering just the products belonging to the 'Dips & Salsa' category
product_dands = product_real[product_real['CATEGORY_2']=='Dips & Salsa']
product_dands.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit
207,Snacks,Dips & Salsa,Hummus,NaN,AMAZON,365 WHOLE FOODS MARKET,723055864228,True
246,Snacks,Dips & Salsa,Ranch Dip,NaN,DUKE CANNON,DUKE CANNON,056360005490,True
280,Snacks,Dips & Salsa,Ranch Dip,NaN,PLACEHOLDER MANUFACTURER,PRIVATE LABEL,081131276458,True
631,Snacks,Dips & Salsa,Hummus,NaN,NaN,NaN,851351007794,True
673,Snacks,Dips & Salsa,Ranch Dip,NaN,NaN,NaN,062908000216,True


In [153]:
#Finding the break down of the missing values of brands in the dips and salsa products
product_dands['BRAND'].isna().value_counts()

,count
BRAND,
False,9939
True,6329


In [154]:
##Finding the break down of the missing values of manufacturers in the dips and salsa products
product_dands['MANUFACTURER'].isna().value_counts()

,count
MANUFACTURER,
False,9939
True,6329


The number of missing values in both are equal.

In [155]:
product_dands[product_dands['MANUFACTURER'].isna() & product_dands['BRAND'].notna()]

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit


There are 6329 products in the "Dips & Salsa" category with missing brand and manufacturer.

**Exploring ways to impute the missing brand values with some assumption**

**Way 1:** \
Steps: \
1) We can divide the whole dataset which contains only the products in the Dips and Salsa category into two, one with missing brand and the other with the brand present. \
2) Then we merge (inner join) the transaction data set with both the data sets seperately over the common barcodes between the transaction dataset and both the tables respectively while excluding the null values. separately. Thus two new data sets are formed because of the two merges. \
3) Once we form them, we check for the common user_ids in both the data sets. One data set has a brand associated to the user_id and the other doesnt have a brand. So based on the assumption explained below, we then perform the imputation of brands based on the common user_ids. \
**Assumption**:  The main assumption here is brand loyalty. It is assumed that for a particular category (Dips and Salsa in this case), a user will always choose a specific brand every time they shop even thuogh the products are different with different barcodes. So by finding the common users, with this assumption we can impute the missing brand values in one dataset with the present values in the other corresponding to a common user. \
4) Then we first impute the brand names in the merged data set containing the missing brand values and then impute the missing brand names in the original dataset with the help of the imputed dataset through the corresponding barcodes.


Let us create two dataframes, one where the brand is missing and one where it is not missing.

In [156]:
dands_brand_isna = product_dands[product_dands['BRAND'].isna()]
dands_brand_nona = product_dands[~(product_dands['BRAND'].isna())]

In [157]:
dands_brand_isna.shape

(6329, 8)

In [158]:
dands_brand_nona.head(3)

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit
207,Snacks,Dips & Salsa,Hummus,NaN,AMAZON,365 WHOLE FOODS MARKET,723055864228,True
246,Snacks,Dips & Salsa,Ranch Dip,NaN,DUKE CANNON,DUKE CANNON,056360005490,True
280,Snacks,Dips & Salsa,Ranch Dip,NaN,PLACEHOLDER MANUFACTURER,PRIVATE LABEL,081131276458,True


In [159]:
#Counting the number missing barcode values in the dataset with missing brand
dands_brand_isna['BARCODE'].isna().value_counts()

,count
BARCODE,
False,6318
True,11


In [160]:
#Counting the number missing barcode values in the dataset with known brands
dands_brand_nona['BARCODE'].isna().value_counts()

,count
BARCODE,
False,9853
True,86


Let us now see the common barcodes between transactions and the above dataframes - one with missing brand and another with the brand present.

In [161]:
dands_tra_namerge = pd.merge(transact_final[transact_final['BARCODE'].notna()], dands_brand_isna[dands_brand_isna['BARCODE'].notna()], left_on='BARCODE', right_on='BARCODE', how='inner')
dands_tra_namerge.head(3)


,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,barcode_is_12_digit
0,a75e33ac-5f88-4444-b0ca-2a4cac0ff352,2024-08-23,2024-08-27 12:33:57.245000+00:00,JEWEL OSCO,5c16a0ceefb01c1db214e39b,810041599056,1.0,5.99,Snacks,Dips & Salsa,Guacamole,NaN,NaN,NaN,True
1,eac76139-1799-47ff-8511-b0d08dcfcbca,2024-06-27,2024-07-08 08:34:26.053000+00:00,WALMART,61e8cd09713c8927067a22a3,781138703161,1.0,2.58,Snacks,Dips & Salsa,Salsa,NaN,NaN,NaN,True
2,b5a8671e-4bf4-46e6-99e7-7616e7267f81,2024-07-21,2024-07-21 10:40:08.690000+00:00,SAFEWAY,6404c82c2a703a195927b4cb,852660005143,1.0,6.49,Snacks,Dips & Salsa,Salsa,NaN,NaN,NaN,True


In [162]:
dands_tra_namerge.shape

(21, 15)

In [163]:
dands_tra_namerge['BARCODE'].value_counts()

,count
BARCODE,
781138801157,3
781138703161,2
781138700160,2
810041590640,2
810041599056,1
852660005143,1
897194002183,1
795631809091,1
049733090179,1


In [164]:
dands_tra_nonamerge = pd.merge(transact_final[transact_final['BARCODE'].notna()], dands_brand_nona[dands_brand_nona['BARCODE'].notna()], left_on='BARCODE', right_on='BARCODE', how='inner')
dands_tra_namerge.head(3)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,barcode_is_12_digit
0,a75e33ac-5f88-4444-b0ca-2a4cac0ff352,2024-08-23,2024-08-27 12:33:57.245000+00:00,JEWEL OSCO,5c16a0ceefb01c1db214e39b,810041599056,1.0,5.99,Snacks,Dips & Salsa,Guacamole,NaN,NaN,NaN,True
1,eac76139-1799-47ff-8511-b0d08dcfcbca,2024-06-27,2024-07-08 08:34:26.053000+00:00,WALMART,61e8cd09713c8927067a22a3,781138703161,1.0,2.58,Snacks,Dips & Salsa,Salsa,NaN,NaN,NaN,True
2,b5a8671e-4bf4-46e6-99e7-7616e7267f81,2024-07-21,2024-07-21 10:40:08.690000+00:00,SAFEWAY,6404c82c2a703a195927b4cb,852660005143,1.0,6.49,Snacks,Dips & Salsa,Salsa,NaN,NaN,NaN,True


In [165]:
dands_tra_nonamerge.shape

(305, 15)

In [166]:
# Merging the above two datasets to find the common user_ids
dands_nona_na_merge =  pd.merge(dands_tra_nonamerge[dands_tra_nonamerge['BARCODE'].notna()],
                                dands_tra_namerge[dands_tra_namerge['BARCODE'].notna()], left_on='USER_ID', right_on='USER_ID', how='inner')

In [167]:
dands_nona_na_merge.shape

(1, 29)

Bad News! There is only one common user_id between both the merged datasets. Therefore, we can impute only one missing brand.

Good news is that there is another approach we can try!

**Way 2:** \
Steps: \
1) Firstly, we check for the brand breakdown in the Dips and Salsa data set containing the product details with the known brands. \
2) We then choose one brand and check the details of the products in that brand. \
**Aim**: We are looking for some pattern in the products of the same brand. For example, all the barcodes of a particular brand in the dips and salsa category might have the same last 4 digits.  \
3) Once we find a pattern, we look for matches in the pattern in the other dataset with the missing brand values keeping in mind the following assumption. \
**Assumption**: It is assumed that all the products with an identical barcode pattern belong to the same brand for a given category. \
4) Keeping the above assumption in mind, we impute the missing brands in the original dataset with the brands names of those products whose barcode patterns match with those of the products with missing brand names belonging to the 'Dips and Salsa' category.   


In [168]:
# Checking the list of brands and their frequency
dands_brand_nona['BRAND'].value_counts().head(30)

,count
BRAND,
PRIVATE LABEL,1166
SABRA,268
WHOLLY,217
TOSTITOS,149
PACE,119
GOOD FOODS,115
MEIJER,112
MARZETTI,109
BRAND NOT KNOWN,108


In [169]:
#Let us check the details of the products belonging to the "WHOLLY" brand
n = dands_brand_nona[dands_brand_nona['BRAND']=='WHOLLY']
n.head(10)

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit
1374,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112279526,True
4641,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112681510,True
5781,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,016112039210,True
10458,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112027868,True
11391,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112329535,True
13738,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112028551,True
18343,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112031544,True
22085,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,700718414648,True
28261,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112029381,True
29203,Snacks,Dips & Salsa,Guacamole,NaN,HORMEL FOODS,WHOLLY,616112031988,True


It can be seen that, most of the barcode's first 6 digits are mostly identical!

In [170]:
result_n = n[(n['BARCODE'].str.contains('616112', case=False, na=False)) |
            (n['BARCODE'].str.contains('016112', case=False, na=False))]
result_n.shape

(201, 8)

In [171]:
n.shape

(217, 8)

Out of 217 items, 201 items' barcodes' first 6 digits match! There is a clear pattern! In the above case, two patterns.

Let us check for another brand

In [172]:
m = dands_brand_nona[dands_brand_nona['BRAND']=='SABRA']
m.shape

(268, 8)

In [173]:
m.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit
1337,Snacks,Dips & Salsa,Hummus,NaN,PEPSICO,SABRA,040822344708,True
1573,Snacks,Dips & Salsa,Hummus,NaN,PEPSICO,SABRA,700720008569,True
4961,Snacks,Dips & Salsa,Hummus,NaN,PEPSICO,SABRA,040822433655,True
7093,Snacks,Dips & Salsa,Hummus,NaN,PEPSICO,SABRA,040822344043,True
16050,Snacks,Dips & Salsa,Hummus,NaN,PEPSICO,SABRA,040822012157,True


In [174]:
result_m = m[(m['BARCODE'].str.contains('040822', case=False, na=False))]
result_m.shape

(224, 8)

Out of 268 items, the barcodes of 224 items in that list have the same series of digits.

In [175]:
# Checking for the pattern match in the dataset with missing brand names
result_m2 = dands_brand_isna[(dands_brand_isna['BARCODE'].str.contains('040822', case=False, na=False))]
result_m2.shape

(1, 8)

In [176]:
result_m2

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE,barcode_is_12_digit
156575,Snacks,Dips & Salsa,Salsa,NaN,NaN,NaN,749404082228,True


Only one product's barcode pattern matches the common pattern. Nonetheless it is a good news! Because, this match is just for one brand with one barcode pattern. It can be assumed that there are many such groups present following different fixed patterns. If we can match all the products in the dips and salsa category according to the barcode patterns and impute the missing brand names with the corresponding brand, then we might considerbly reduce the missing values!

In [177]:
# Function to fill missing brands based on barcode prefixes for the rows with 'CATEGORY_2' = 'Dips & Salsa'
def fill_missing_brands(product_final):
    # Create a dictionary to store the first 6 characters of barcode and their corresponding brand (only for 'Dips & Salsa' rows)
    barcode_to_brand = {}

    # Iterate over the DataFrame and populate the barcode_to_brand dictionary with non-missing brands for 'Dips & Salsa' rows
    for index, row in product_final.iterrows():
        if pd.notnull(row['BARCODE']) and pd.notnull(row['BRAND']) and row['CATEGORY_2'] == 'Dips & Salsa':
            barcode_prefix = row['BARCODE'][:6]  # Get first 6 characters of the barcode
            barcode_to_brand[barcode_prefix] = row['BRAND']

    # Now, fill missing Brand values based on barcode_prefix matching (consider only non-null barcodes and 'Dips & Salsa' rows)
    for index, row in product_final.iterrows():
        if pd.isnull(row['BRAND']) and pd.notnull(row['BARCODE']) and row['CATEGORY_2'] == 'Dips & Salsa':
            barcode_prefix = row['BARCODE'][:6]
            if barcode_prefix in barcode_to_brand:
                product_final.at[index, 'BRAND'] = barcode_to_brand[barcode_prefix]  # Fill missing brand with the known brand

    return product_final

# Fill missing brands
product_filled = fill_missing_brands(product_final)

product_filled['BRAND'].isna().value_counts()


,count
BRAND,
False,620885
True,224610


In [178]:
product_filled.shape

(845495, 7)

In [179]:
product_filled['BRAND'].isna().value_counts()

,count
BRAND,
False,620885
True,224610


In [180]:
product_filled.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,796494407820
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,023278011028
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,461817824225
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,035000466815
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,0806810850459


In [181]:
# Extracting the products in the Dips and Salsa category
product_dands2 = product_filled[product_filled['CATEGORY_2']=='Dips & Salsa']
product_dands2.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
207,Snacks,Dips & Salsa,Hummus,NaN,AMAZON,365 WHOLE FOODS MARKET,723055864228
246,Snacks,Dips & Salsa,Ranch Dip,NaN,DUKE CANNON,DUKE CANNON,056360005490
280,Snacks,Dips & Salsa,Ranch Dip,NaN,PLACEHOLDER MANUFACTURER,PRIVATE LABEL,081131276458
631,Snacks,Dips & Salsa,Hummus,NaN,NaN,NaN,851351007794
673,Snacks,Dips & Salsa,Ranch Dip,NaN,NaN,NaN,062908000216


In [182]:
product_dands2['BRAND'].isna().value_counts()

,count
BRAND,
False,11801
True,4467


It can be seen from the above result that we were able to reduce considerable amount of missing values!

In [183]:
product_final2 = product_filled
product_final2.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,796494407820
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,023278011028
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,461817824225
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,035000466815
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,0806810850459


**Let us now check investigate different merges of all the three tables.**

Users and Transactions

In [184]:
users_transact_inner = pd.merge(transact_final, users_final, left_on='USER_ID', right_on='ID', how='inner')
users_transact_inner.shape

(130, 14)

There are only 130 user_ids which are common to both the tables.

In [185]:
users_transact_left = pd.merge(transact_final, users_final, left_on='USER_ID', right_on='ID', how='left')
users_transact_left.shape

(24852, 14)

***Data Quality Issue*** \
It can be seen that there are plenty of users who made purchases but their records are missing from the users table. This  would adversely affect the business analyses which are drawn from the users data based on the transactions made because, a lot of valid transactions would go unaccounted for because of the missing users data!

In [186]:
users_transact_right = pd.merge(transact_final, users_final, left_on='USER_ID', right_on='ID', how='right')
users_transact_right.shape

(100038, 14)

Products and Transactions

Firstly, due to the presence of missing values and repeated barcode values in both the columns, the analyses could be faulty.

In [187]:
product_transact_inner = pd.merge(transact_final[transact_final['BARCODE'].notna()],
                                  product_final2[product_final2['BARCODE'].notna()], left_on='BARCODE', right_on='BARCODE', how='inner')
product_transact_inner.shape

(12363, 14)

***Data Quality Issue*** \
More than half of the transactions have been missed after the join. This means that there are a lot of products that were bought that didnt have records in the products table.

In [188]:
product_transact_left = pd.merge(transact_final[transact_final['BARCODE'].notna()],
                                  product_final2[product_final2['BARCODE'].notna()], left_on='BARCODE', right_on='BARCODE', how='left')
product_transact_left.shape

(21996, 14)

In [189]:
product_transact_right = pd.merge(transact_final[transact_final['BARCODE'].notna()],
                                  product_final2[product_final2['BARCODE'].notna()], left_on='BARCODE', right_on='BARCODE', how='right')
product_transact_right.shape

(847329, 14)

# Downloading the files to my computer

In [190]:
from google.colab import files

In [191]:
users_final.to_csv('users_final.csv', index=False)
transact_final.to_csv('transact_final.csv', index=False)
product_final2.to_csv('product_final2.csv', index=False)


In [192]:
files.download('users_final.csv')
files.download('transact_final.csv')
files.download('product_final2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>